In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import *
from collections import Counter

In [6]:
class KNN:

    def __init__(self):
        self.ACs = []
        for ac_name in os.listdir('./ACs'):
            ac_i = pd.read_pickle('./ACs/'+ac_name)
            self.ACs.append(ac_i)
        self.n_rows = len(self.ACs[0])

    def get_X_Y(self, row: int) -> (pd.DataFrame, pd.DataFrame):
        data = []
        for AC in self.ACs:
            AC = pd.DataFrame(AC)
            data.append(AC.iloc[row,:].tolist())
        data = pd.DataFrame(data).reset_index(drop=True)
        X = data.iloc[:,1:]
        Y = data.iloc[:,0]
        
        return X, Y.astype(int).tolist()   
    
    def cluster(self, x_row: pd.DataFrame) -> (list):
                
        nbrs = NearestNeighbors(n_neighbors=len(x_row)//2, algorithm='brute', metric='manhattan').fit(x_row)
        distances, indices = nbrs.kneighbors(x_row)
        
        labbels = distances.mean(axis=1)
                    
        anomaly_score = (labbels - min(labbels)) / (max(labbels) - min(labbels))
        
        return anomaly_score.tolist()
    
    def cluster_all(self, do_score=False) -> (list, list):
        
        anomaly_score, y = [], []
        
        for row in range(self.n_rows):
            print(row)
            x_row, y_row = self.get_X_Y(row)
            if sum(y_row) >= len(y_row)//2: 
                print('oops')

            anomaly_score_row = self.cluster(x_row)
            anomaly_score += anomaly_score_row
            y += y_row
                    
        return anomaly_score, y
    
    def auc_score(self, anomaly_score: list, y: list):
        # AUC
        fpr, tpr, thresholds = roc_curve(y, anomaly_score)
        AUC = auc(fpr, tpr)
        print('AUC score: %f' % AUC)
    
    def score(self, anomaly_score: list, y: list, threashold=0.8) -> int:
        
        yhat = [ 1 if y_i >= threashold else 0 for y_i in anomaly_score]
        
        # accuracy: (tp + tn) / (p + n)
        accuracy = accuracy_score(y, yhat)
        print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(y, yhat)
        print('Precision: %f' % precision)
        # recall: tp / (tp + fn) More important in this case
        recall = recall_score(y, yhat)
        print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y, yhat)
        print('F1 score: %f' % f1)
        return f1

In [7]:
knn = KNN()

anomaly_score, y = knn.cluster_all()
knn.auc_score(anomaly_score, y)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
AUC score: 0.965104
0.5
Accuracy: 0.844000
Precision: 0.204082
Recall: 1.000000
F1 score: 0.338983
0.6
Accuracy: 0.866000
Precision: 0.229885
Recall: 1.000000
F1 score: 0.373832
0.7
Accuracy: 0.888000
Precision: 0.263158
Recall: 1.000000
F1 score: 0.416667
0.8
Accuracy: 0.902000
Precision: 0.289855
Recall: 1.000000
F1 score: 0.449438
0.9
Accuracy: 0.912000
Precision: 0.312500
Recall: 1.000000
F1 score: 0.476190
